## Ejercicio 01

1. Obtener todas las cartas de la API.
1. La key `colors` contiene un value que es una lista. Filtrar las cartas que contengan `"Green"` en esa lista.
1. Generar una base de datos con SQLAlchemy para almacenar estas cartas.
1. Las columnas que debemos crear son (entre paréntesis está el nombre de la key que tienen en el diccionario):
    * nombre (`name`)
    * ** multiverse_id (`multiverseid`)  <== queda eliminado a menos que queraís hacer la parte extra
    * url_imagen (`imageUrl`) || tipo (`type`)
    * rareza (`rarity`)
    * **Extra**: en las cartas verdes que NO tienen `multiverseid`, crearlo y darle el valor `0`
    
1. De la lista de cartas filtradas que hemos obtenido en el punto $2$, guardarlas todas en la base de datos.

**Extra 🔥**

* Crear una función que obtenga la información de la carta de la base de datos en base a un `id` o como queráis.
* En las cartas que no tengan la variable `"imageUrl"`, crearla y poner: http://placegoat.com/200/200
* Descargar la imagen de esta carta (o de la cabra 😂) y guardarla en un archivo en el disco.
* Crear un archivo JSON que sea una lista de diccionarios con el siguiente formato:

```python
[
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    .
    .
    .
]
```

Por ejemplo:

```python
req_img = requests.get(v[0]["imageUrl"])

with open("imagen.jpg", "wb") as f:
    f.write(req_img.content)
```

In [5]:
import json

with open("cartas.json", "w") as f:
    f.write(json.dumps(cartas))

In [12]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

import json
import requests

In [3]:
# import requests

# r = requests.get("https://api.magicthegathering.io/v1/cards")

# data = r.json()
# cartas = data["cards"]

In [2]:
with open("cartas.json") as f:
    cartas = json.loads(f.read())

In [3]:
engine = create_engine("sqlite:///cartas_url.db", echo=True)

Base = declarative_base()


class Verde(Base):
    
    __tablename__ = "verdes"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    # multiverse_id = Column(Integer)
    url = Column(String)
    
    rarity = Column(String)

    def __repr__(self):
        # url='{self.url}',
        return f"<Verde(name='{self.name}', rarity='{self.rarity}')>"

In [4]:
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()

2020-07-10 10:10:08,643 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-10 10:10:08,647 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 10:10:08,649 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-10 10:10:08,650 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 10:10:08,652 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("verdes")
2020-07-10 10:10:08,655 INFO sqlalchemy.engine.base.Engine ()


In [5]:
verdes = []

for carta in cartas:

    colores = carta["colors"]

    if "Green" in colores:
        try:
            url_imagen = carta["imageUrl"]
        except KeyError:  # no tiene la key ["imageUrl"]
            url_imagen = "http://placegoat.com/200/200"

        nombre = carta["name"]
        rareza = carta["rarity"]

        carta_nueva = Verde(name=nombre, url=url_imagen, rarity=rareza)
        verdes.append(carta_nueva)

In [6]:
verdes

[<Verde(name='Abundance', rarity='Rare')>,
 <Verde(name='Aggressive Urge', rarity='Common')>,
 <Verde(name='Avatar of Might', rarity='Rare')>,
 <Verde(name='Avatar of Might', rarity='Rare')>,
 <Verde(name='Birds of Paradise', rarity='Rare')>,
 <Verde(name='Birds of Paradise', rarity='Rare')>,
 <Verde(name='Blanchwood Armor', rarity='Uncommon')>,
 <Verde(name='Blanchwood Armor', rarity='Uncommon')>,
 <Verde(name='Canopy Spider', rarity='Common')>,
 <Verde(name='Canopy Spider', rarity='Common')>,
 <Verde(name='Civic Wayfinder', rarity='Common')>,
 <Verde(name='Commune with Nature', rarity='Common')>,
 <Verde(name='Craw Wurm', rarity='Common')>,
 <Verde(name='Creeping Mold', rarity='Uncommon')>]

In [7]:
session.add_all(verdes)

In [8]:
session.commit()

2020-07-10 10:10:12,189 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-10 10:10:12,195 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (name, url, rarity) VALUES (?, ?, ?)
2020-07-10 10:10:12,199 INFO sqlalchemy.engine.base.Engine ('Abundance', 'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=130483&type=card', 'Rare')
2020-07-10 10:10:12,202 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (name, url, rarity) VALUES (?, ?, ?)
2020-07-10 10:10:12,203 INFO sqlalchemy.engine.base.Engine ('Aggressive Urge', 'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=130525&type=card', 'Common')
2020-07-10 10:10:12,204 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (name, url, rarity) VALUES (?, ?, ?)
2020-07-10 10:10:12,205 INFO sqlalchemy.engine.base.Engine ('Avatar of Might', 'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=135249&type=card', 'Rare')
2020-07-10 10:10:12,207 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

import json

In [10]:
carta = session.query(Verde).filter(Verde.name == "Craw Wurm").first()

2020-07-10 10:10:25,965 INFO sqlalchemy.engine.base.Engine SELECT verdes.id AS verdes_id, verdes.name AS verdes_name, verdes.url AS verdes_url, verdes.rarity AS verdes_rarity 
FROM verdes 
WHERE verdes.name = ?
 LIMIT ? OFFSET ?
2020-07-10 10:10:25,968 INFO sqlalchemy.engine.base.Engine ('Craw Wurm', 1, 0)


In [11]:
carta.url

'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=130527&type=card'

In [14]:
resultado = requests.get(carta.url)

with open("imagen_carta.jpg", "wb") as f:
    f.write(resultado.content)

In [17]:
def descargar_imagen_carta(nombre_carta: str, nombre_archivo: str) -> bool:
    """Descarga la imagen de un carta haciendo la query previamente."""
    
    carta = session.query(Verde).filter(Verde.name == nombre_carta).first()
    
    resultado = requests.get(carta.url)

    with open(nombre_archivo, "wb") as f:
        f.write(resultado.content)
        
    return bool

In [18]:
descargar_imagen_carta(nombre_carta="Commune with Nature", nombre_archivo="comune_nature.jpg")

2020-07-10 10:14:39,090 INFO sqlalchemy.engine.base.Engine SELECT verdes.id AS verdes_id, verdes.name AS verdes_name, verdes.url AS verdes_url, verdes.rarity AS verdes_rarity 
FROM verdes 
WHERE verdes.name = ?
 LIMIT ? OFFSET ?
2020-07-10 10:14:39,092 INFO sqlalchemy.engine.base.Engine ('Commune with Nature', 1, 0)


bool

In [13]:
# carta[0].url

'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=130527&type=card'

In [ ]:
for user in (
    session.query(User)
    .filter(User.name == "daniel")
    .filter(User.fullname == "daniel orejuela")
):
    print(user)

In [ ]:
for nombre, nombre_completo in session.query(User.name, User.fullname):
    print(nombre, nombre_completo)